In [1]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.8 MB/s eta 0:00:00


In [1]:
import gradio as gr
import pandas as pd
from torch import nn
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.metrics import f1_score
import torch
import nltk
nltk.download(['punkt', 'stopwords'])
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def remove_short_strings(df:pd.DataFrame, string_column:str)->pd.DataFrame:
    df[string_column] = df[string_column].astype(str)
    df['length'] = df[string_column].str.len()
    df = df.drop(df[df['length'] == 1].index)
    df = df.drop(columns=['length'])
    return df
def remove_one_character_words(row):
    words = row['text'].split()
    return ' '.join([word for word in words if len(word) > 1])
def ret_list_to_str(liste):
  return " ".join (i for i in liste)
def preprocess_tweet(tweet):
    # Convert to lower case
    tweet = tweet.lower() 
    # Replace repeating characters
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    # Remove non-Turkish characters
    tweet = re.sub(r'[^a-zA-ZçÇğĞıİöÖşŞüÜ\s]', '', tweet)
    # Remove extra whitespaces
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet
def cleaning_stopwords(text,stop_words):
    return " ".join([word for word in str(text).split() if word not in stop_words])
from nltk.corpus import stopwords
# Türkçe stop words
turkish_stopwords = stopwords.words('turkish')
turkish_stopwords.append("bir")
turkish_stopwords=set(turkish_stopwords)
 ##burada saçma kelimeler var bunu kullanmayalım 


from sklearn import preprocessing
from nltk.tokenize import word_tokenize


def prep_and_sw_and_tokenize(df):

  turkish_stopwords = stopwords.words('turkish')
  turkish_stopwords.append("bir")
  stop_words=set(turkish_stopwords)
  df["text"]=df["text"].apply(preprocess_tweet)
  df['text'] = df["text"].apply(lambda text: cleaning_stopwords(text,stop_words))

  #df['text'] = df.apply(remove_one_character_words, axis=1)


  return df

In [3]:

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
class BertClassifierConv1D(nn.Module):
    def __init__(self, dropout=0.5, num_classes=5):
        super(BertClassifierConv1D, self).__init__()
        
        self.bert = BertModel.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', return_dict=True)
        self.conv1d = nn.Conv1d(in_channels=self.bert.config.hidden_size, out_channels=128, kernel_size=5)
        self.bilstm = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(128, num_classes)

    def forward(self, input_id, mask):
        output = self.bert(input_ids=input_id, attention_mask=mask).last_hidden_state
        output = output.permute(0, 2, 1) # swap dimensions to prepare for Conv1d layer
        output = self.conv1d(output)
        output, _ = self.bilstm(output.transpose(1, 2))
        output = self.dropout(output)
        output = self.linear(output.mean(dim=1))
        return output
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.texts = [tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors="pt") for text in df]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        batch_texts = self.texts[idx]
        return batch_texts
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

    #use_cuda = torch.cuda.is_available()
    #device = torch.device("cuda" if use_cuda else "cpu")
    device= torch.device("cpu")

    #if use_cuda:

     #   model = model.cuda()

    total_acc_test = 0
    output_indices = []
    with torch.no_grad():

        for test_input in test_dataloader:

              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
           

              batch_indices = output.argmax(dim=1).tolist()
              output_indices.extend(batch_indices)

    
   
    return output_indices


In [4]:
def auth(username, password):
    if username == "Hive_Hereos" and password == "Y2IB3HV8GBXED00S":
        return True
    else:
        return False


In [5]:
global model
model =BertClassifierConv1D()

model.load_state_dict(torch.load(r"C:\Users\Asus\Desktop\teknofest\gradio\model_weights_epoch_11.pt", map_location=torch.device('cpu')))


Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [6]:
import logging
logging.basicConfig(filename=r'C:\Users\Asus\Desktop\teknofest\gradio\app.log', filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO)


def predict(df):
    # TODO:
    df["offensive"] = 1 
    df["target"] = None
    # ***************************
    try:
      # WRITE YOUR INFERENCE STEPS BELOW  # HERE
      df=prep_and_sw_and_tokenize(df)
      #df.to_csv("preprocess.csv", index=False, sep="|")
      labels = {'INSULT':0,
            'OTHER':1,
            'PROFANITY':2,
            'RACIST':3,
            'SEXIST':4
            }
      logging.info("Başlıyoruz")
     
      logging.info("Model yüklendi")
      logging.info(df.text)
      a=evaluate(model, df["text"])
      
      test_labels=[]
      for number in a:
          label = list(labels.keys())[list(labels.values()).index(number)]  # Sayıyı etikete dönüştürüyoruz.
          test_labels.append(label)  # Yeni etiketi listeye ekliyoruz.
      df["target"]=test_labels
      logging.info(f"Test Labels: {test_labels}")
      for index, row in df.iterrows():
        if row['target'] == 'OTHER':
          df.at[index, 'offensive'] = 0
      df.to_csv("end_of_code.csv", index=False, sep="|")
    except Exception as e:
      logging.error("Error occurred", exc_info=True)   
      raise e
    #
    # *********** END ***********


    return df


In [7]:
def get_file(file):
    output_file = "output_Hive_Hereos.csv"

    # For windows users, replace path seperator
    file_name = file.name.replace("\\", "/")

    df = pd.read_csv(file_name, sep="|")

    predict(df)
    df.to_csv(output_file, index=False, sep="|")
    return (output_file)


In [ ]:
# Launch the interface with user password
iface = gr.Interface(get_file, "file", "file")

if __name__ == "__main__":
    iface.launch(share=True, auth=auth,debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d7857c2e67677f4433.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [32]:
iface.close()

Closing server running on port: 7860


In [82]:
df = pd.read_csv(r"C:\Users\Asus\Desktop\teknofest\codes\datas\teknofest_train_final.csv", sep="|")
df=df[1:16]
df=df[["id","text"]]
df.to_csv(r"C:\Users\Asus\Desktop\teknofest\codes\datas\deneme_test.csv", index=False, sep="|")

In [ ]:
import session_info
session_info.show()